In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/My \Drive/Sem \8 \files/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/
/content/drive/My Drive/Sem 8 files
 data				  Singapore_net.tntp
 euclidean_distances.json	  Singapore_node.tntp
 Frank_Wolfe.py			  Singapore_trips_1000.tntp
'Fri Feb 14 09_06_16 2020.json'   Singapore_trips_275.tntp
 __pycache__			  Singapore_trips_2884.tntp
 Shortest_path_lengths.pkl	  Singapore_trips_70.tntp
 Singapore_net_high.tntp	  TransportationNetworks.py
 Singapore_net_mod.tntp		  utils.py
 Singapore_net_off.tntp


In [0]:
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
import math
import time

import TransportationNetworks as tn
from networkx.algorithms.flow import edmonds_karp
from heapq import heappush, heappop
from itertools import count

In [0]:
class Run:
    def __init__(self, link_file, trip_file, node_file, SO=True):
        self.SO = SO
        
        nw = tn.Network(link_file, trip_file, node_file, self.SO)
        self.od_vols = nw.od_vols

        ## initialization
        self.network = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
                   'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
                   'auxiliary':[], 'cost':[]} for (u, v, d) in nw.graph.edges(data=True)}
        
        self.fwResult = {'theta':[], 'z':[]}
        
        nw.all_or_nothing_assignment()
        nw.update_linkcost()
        
        for linkKey, linkVal in self.network.items():
            linkVal['cost'].append(nw.graph[linkKey[0]][linkKey[1]]['weight'])
            linkVal['auxiliary'].append(nw.graph[linkKey[0]][linkKey[1]]['object'].vol)
            linkVal['flow'].append(nw.graph[linkKey[0]][linkKey[1]]['object'].vol)
            
        ## iterations
        iterNum=0
        iteration = True
        start = time.time()
        PERIOD_OF_TIME = 900 # 15min

        while iteration:
            iterNum += 1
            nw.all_or_nothing_assignment()
            nw.update_linkcost()
            
            for linkKey, linkVal in self.network.items():
                linkVal['auxiliary'].append(nw.graph[linkKey[0]][linkKey[1]]['object'].vol)
                
            theta = self.lineSearch()
            self.fwResult['theta'].append(theta)
            
            for linkKey, linkVal in self.network.items():
                aux = linkVal['auxiliary'][-1]
                flow = linkVal['flow'][-1]
                linkVal['flow'].append(flow + theta*(aux-flow))
                
                nw.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
                nw.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
                
            
            nw.update_linkcost()
            
            z=0
            for linkKey, linkVal in self.network.items():
                linkVal['cost'].append(nw.graph[linkKey[0]][linkKey[1]]['weight'])
                totalcost = nw.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
                z+=totalcost
                
            self.fwResult['z'].append(z)        
            
            if iterNum == 1:
                iteration = True
            else:
                if abs(self.fwResult['z'][-2] - self.fwResult['z'][-1]) <= 0.002 or \
                   iterNum==1000 or \
                   time.time() > start + PERIOD_OF_TIME:
                   iteration = False
                
        print(iterNum, abs(self.fwResult['z'][-2] - self.fwResult['z'][-1]))
        self.graph = nw.graph
                    
    def BPR(self, t0, xa, ca, alpha, beta):
        ta = t0 * (1 + alpha * pow((xa/ca), beta))
        return ta
    
    def calculateZ(self, theta):
        z = 0
        for linkKey, linkVal in self.network.items():
            t0 = linkVal['t0']
            ca = linkVal['capa']
            beta = linkVal['beta']
            alpha = linkVal['alpha']
            aux = linkVal['auxiliary'][-1]
            flow = linkVal['flow'][-1]
            
            if SO == False:
                z += integrate.quad(lambda x: self.BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
            elif SO == True:
                z += list(map(lambda x : x * self.BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
        return z
    
    def lineSearch(self):
        theta = minimize_scalar(lambda x: self.calculateZ(x), bounds = (0,1), method = 'Bounded')
        return theta.x


    def shortest_successive_path(self, source, target):
        if target == source:
            return [target]

        paths = {source: [source], target: []} # dictionary of paths
        G_succ = self.graph._succ
        push = heappush
        pop = heappop        
        dist = {}  # dictionary of final width
        c = count() # use the count c to avoid comparing nodes
        fringe = [] # fringe is heapq with 3-tuples (distance,c,node)

        for n in self.graph.nodes:
            dist[n] = float('inf')
        dist[source] = 0
        
        push(fringe, (dist[source], next(c), source))
        while fringe:
            (w, _, v) = pop(fringe)
            if v == target:
                break

            for u, e in G_succ[v].items():
                # Check for only those edges who have enough capacity left
                if e['capacity'] > 0:
                    dist_vu = e['weight']
                    alt = dist[v] + dist_vu

                    if alt < dist[u]:
                        dist[u] = alt
                        push(fringe, (dist[u], next(c), u))
                        paths[u] = paths[v] + [u]

        return paths[target]

    
    def showODPath(self):
        """
        Method for presenting table of the optimal traffic assignment of the Frank-Wolfe algorithm procedure
        """
        capacity = dict()
        for (u, v, d) in self.graph.edges(data=True):
            capacity[(u,v)] = math.ceil(d['object'].vol)

        nx.set_edge_attributes(self.graph, capacity, name='capacity')

        # sort OD pairs according to least path options first
        x = dict()
        for (origin, dest), demand in self.od_vols.items():
            if demand != 0:
                path = nx.dijkstra_path(self.graph, origin, dest, weight="length")

                neighbours = 0
                for p in path[:-1]:
                    neighbours += len(list(nx.neighbors(self.graph, p)))

                x[(origin, dest, demand)] = neighbours

        OD = sorted(x.items(), key = lambda kv:(kv[1], kv[0]))

        # Decomposing flow into a path for every request
        infeasible = dict()
        cost = 0

        for (origin, dest, demand), _ in OD:
            while demand > 0:
                path = self.shortest_successive_path(origin, dest)

                if path == []: # Add to waiting queue
                    infeasible[(origin, dest)] = demand
                    break
                else:
                    # Decrement capacity of chosen path by 1
                    for i in range(len(path)-1):
                        u = path[i]
                        v = path[i+1]
                        self.graph[u][v]['capacity'] -= 1
                        cost += self.graph[u][v]['weight']
                    demand = demand - 1                   

        count = 0
        for d in infeasible.values():
            count += d
        print("Number of infeasible trips:", count)
        print("Cost of system:", cost)

        # Route infeasible paths in a user equilibrium way
        if len(infeasible) > 0:
            new_capacity = dict()

            for (u, v, d) in self.graph.edges(data=True):
                new_capacity[(u,v)] = d['object'].capacity -\
                                      math.ceil(d['object'].vol) +\
                                      d['capacity']

            nx.set_edge_attributes(self.graph, new_capacity, name='capacity')
            
            count = 0
            for (origin, dest), demand in infeasible.items():
                while demand > 0:
                    path = self.shortest_successive_path(origin, dest)

                    if path == []: # Add to waiting queue
                        count += demand
                        break
                    else:
                        # Decrement capacity of chosen path by 1
                        for i in range(len(path)-1):
                            u = path[i]
                            v = path[i+1]
                            self.graph[u][v]['capacity'] -= 1
                            cost += self.graph[u][v]['weight']
                        demand = demand - 1 

            print("Number of infeasible trips after residual allocation:", count)
            print("New cost of system after CSO:", cost)

        print("DONE!")
        return cost

In [0]:
node_file = 'Singapore_node.tntp'
trip_file = 'Singapore_trips_1000.tntp'

## Off-peak

In [0]:
link_file = 'Singapore_net_off.tntp'

In [6]:
SO = True
fw = Run(link_file, trip_file, node_file, SO)
se_off = fw.showODPath()

41 5499.746619064361
Number of infeasible trips: 31.0
Cost of system: 15464345.197814113
Number of infeasible trips after residual allocation: 31.0
New cost of system after CSO: 15464345.197814113
DONE!


In [7]:
SO = False
fw = Run(link_file, trip_file, node_file, SO)
ue_off = fw.showODPath()

31 6119.3501063250005
Number of infeasible trips: 49.0
Cost of system: 15378675.74622567
Number of infeasible trips after residual allocation: 46.0
New cost of system after CSO: 15422945.901433166
DONE!


In [8]:
print(se_off, ue_off)

15464345.197814113 15422945.901433166


## Mod-peak

In [0]:
link_file = 'Singapore_net_mod.tntp'

In [10]:
SO = True
fw = Run(link_file, trip_file, node_file, SO)
se_mod = fw.showODPath()

KeyboardInterrupt: ignored

In [0]:
SO = False
fw = Run(link_file, trip_file, node_file, SO)
ue_mod = fw.showODPath()

## High-peak

In [0]:
link_file = 'Singapore_net_high.tntp'

In [0]:
SO = True
fw = Run(link_file, trip_file, node_file, SO)
se_high = fw.showODPath()

In [0]:
SO = False
fw = Run(link_file, trip_file, node_file, SO)
ue_high = fw.showODPath()

## Plot!

In [0]:
system = [se_off, se_mod, se_high] 
user = [ue_off, ue_mod, ue_high] 

labels = ["Off-Peak(12:00 A.M.)", "Moderately-Peak(3:00 P.M.)", "High-Peak(6:00 P.M.)"]
x = np.arange(len(labels))  # the label locations
width = 0.2  # the width of the bars

def autolabel(rects, ax, xpos='center'):
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(math.ceil(height)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha='center', va='bottom')

fig1, ax1 = plt.subplots(figsize=(10,6))
rects2 = ax1.bar(x - width/2, system, width, label='System Equilibrium')
rects1 = ax1.bar(x + width/2, user, width, label='User Equilibrium')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax1.set_ylabel('Total network cost (in seconds)')
ax1.set_xlabel('Customer demand size (1000 trips)')
ax1.set_title('Total network cost in System Equilibrium VS User Equilibrium')
ax1.set_xticks(x)
ax1.set_xticklabels(labels)
ax1.legend()

autolabel(rects1, ax1)
autolabel(rects2, ax1)
fig1.tight_layout()